### ATP Top Rivalry (2000-2019)  
##### Analysis of top rivalries in the ATP circuit for the past decade

In [28]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

#### Create single data file for years 2000-2020

In [7]:
# # create single df for tennis data from 2000-2019
# directory = "/Users/pratikrev/Desktop/Data Mining/Project/Data/Tennis Data - Men's/"

# frames = []

# for filename in sorted(os.listdir(directory)):
#     if filename.endswith((".xls", ".xlsx")):
#         year = os.path.splitext(filename)[0]
        
#         if year != '2020':
#             df = pd.read_excel(directory + filename)

#             # keep only relevant columns
#             columns = ['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
#                        'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1', 'W2', 
#                        'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets','Lsets', 'Comment']
#             df = df[columns]

#             # add year column
#             df['Year'] = year

#             # add to frames
#             frames.append(df)

# tennis_data = pd.concat(frames)

# # store as csv
file_path = "/Users/pratikrev/Desktop/Data Mining/Project/final-project/top-rivalries/TennisData_2000-2019.csv"
# tennis_data.to_csv(file_path, index=False)

In [8]:
# read data
tennis_data = pd.read_csv(file_path)

In [75]:
# top rivalries from 2000-2019

# get head-to-head winner-loser aggregation
playerH2H_win_loss = tennis_data.groupby(["Winner", "Loser"]).agg({'ATP': 'count', 'Year': 'max'}).reset_index()
playerH2H_win_loss.columns = ['Player1', 'Player2', 'Total_Player1_Wins', 'Year']

# get head-to-head loser-winner aggregation
playerH2H_loss_win = tennis_data.groupby(["Loser", "Winner"]).agg({'ATP': 'count', 'Year': 'max'}).reset_index()
playerH2H_loss_win.columns = ['Player1', 'Player2', 'Total_Player2_Wins', 'Year']

# merge data
playerH2H = playerH2H_win_loss.merge(playerH2H_loss_win, on=['Player1', 'Player2'])

playerH2H['H2H'] = playerH2H['Total_Player1_Wins'].astype(str) + " / " + playerH2H['Total_Player2_Wins'].astype(str)
playerH2H['Total'] = playerH2H['Total_Player1_Wins'] + playerH2H['Total_Player2_Wins']

playerH2H['Year'] = np.where(playerH2H['Year_x'] > playerH2H['Year_y'], playerH2H['Year_x'], playerH2H['Year_y'])

# sort player names
playerH2H['Player1'] = np.where(playerH2H['Player1'] < playerH2H['Player2'], playerH2H['Player1'], playerH2H['Player2'])
playerH2H['Player2'] = np.where(playerH2H['Player1'] > playerH2H['Player2'], playerH2H['Player1'], playerH2H['Player2'])
playerH2H['Players'] = playerH2H['Player1'].str.split(" ").str.get(0) + "-" + playerH2H['Player2'].str.split(" ").str.get(0)

# group and sort data
playerH2H =  playerH2H.groupby(['Player1', 'Player2', 'Players', 'H2H']).agg({'Total': 'max', 'Year': 'max'}).reset_index()
playerH2H = playerH2H[playerH2H['Player1'] != playerH2H['Player2']].sort_values(['Total'], ascending=False)

playerH2H['Decade'] = pd.cut(playerH2H.Year.astype(int), [1999, 2010, np.inf], labels=['2000s', '2010s'])

In [76]:
# top 10 rivalries
playerH2H.head(10)Top 

,Player1,Player2,Players,H2H,Total,Year,Decade
3278,Djokovic N.,Nadal R.,Djokovic-Nadal,28 / 24,52,2019,2010s
3250,Djokovic N.,Federer R.,Djokovic-Federer,26 / 21,47,2019,2010s
3804,Federer R.,Nadal R.,Federer-Nadal,17 / 24,41,2019,2010s
3277,Djokovic N.,Murray A.,Djokovic-Murray,25 / 10,35,2017,2010s
3945,Ferrer D.,Nadal R.,Ferrer-Nadal,6 / 26,32,2019,2010s
3830,Federer R.,Wawrinka S.,Federer-Wawrinka,24 / 3,27,2019,2010s
1271,Berdych T.,Djokovic N.,Berdych-Djokovic,3 / 23,26,2017,2010s
7634,Murray A.,Nadal R.,Murray-Nadal,8 / 17,25,2016,2010s
3803,Federer R.,Murray A.,Federer-Murray,14 / 10,24,2015,2010s
3813,Federer R.,Roddick A.,Federer-Roddick,21 / 3,24,2012,2010s


In [ ]:
# Number of finals reached
# title wins
# GS wins
# ranking evolution
# surface perf